# Counter App

## Counter App (Streamlit and sqlite3)

In [ ]:
# Install streamlit
!pip install streamlit -q

In [ ]:
# Restart the notebook

In [ ]:
# Import libraries
import sqlite3
import streamlit as st

In [ ]:
%%writefile app.py
# Import libraries
import streamlit as st
import sqlite3
import os
import time

# Title
st.title("Counter App")
st.caption("""
            Count every request in this app.
            """)
st.subheader("",divider="rainbow")

# Variable
database_name = "counter.db"

# Create a database
if database_name in os.listdir():
    con = sqlite3.connect(database_name)
    cur = con.cursor()
else:
    con = sqlite3.connect(database_name)
    # Database Cursor (cur)
    cur = con.cursor()
    cur.execute("CREATE TABLE counter(hit, time)")
    con.commit()

# Counter
time = time.strftime("Date: %Y-%m-%d | Time: %H:%M:%S UTC")
hit = 1
### Insert into adatabase
cur.execute(f"""
        INSERT INTO counter VALUES
        ("{hit}", "{time}")
        """)
con.commit()

# Total hits
result = cur.execute("""
                    SELECT SUM(hit) 
                    FROM counter
                    """)
st.write(f"### Total hits: **{result.fetchone()[0]}**")

# Current hit
st.write(f"Current: {time}")
# Previous hits
st.subheader("",divider="rainbow")
st.write("### *Previous Hits*")
# Write the data
result = cur.execute("""
                    SELECT * 
                    FROM counter
                    ORDER BY time DESC
                    """)
for name, time in result.fetchall():
    st.text(f"{time}")
    
# Close Connection
con.close()

In [ ]:
%%writefile Dockerfile
FROM python
WORKDIR /app
RUN pip install streamlit -q
COPY app.py .
EXPOSE 8501
CMD ["streamlit", "run", "app.py", "--server.address", "0.0.0.0"]

In [ ]:
# Build an image
!docker build -t app . -q

In [ ]:
# List images
!docker images

In [ ]:
# Run the app in detached mode
!docker run -d -p 8501:8501 -v $(pwd):/app app

In [ ]:
!docker ps -a

In [ ]:
# Remove all docker ps
# !docker rm -f $(docker ps -aq)

In [ ]:
# Create a firewall (GCP)
!gcloud compute --project=$(gcloud config get project) firewall-rules create ports \
    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:8501,tcp:8000,tcp:9000 --source-ranges=0.0.0.0/0

### Add requests

In [ ]:
import time
requests = 50
for _ in range(1,requests+1):
    #
    time.sleep(1)